# 💼 LinkedIn Jobs Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vzucher/brightdata-sdk-python/blob/master/notebooks/04_linkedin_jobs.ipynb)

Analyze job market trends, salaries, and skills demand using LinkedIn data.

## What You'll Learn
1. Searching for jobs by keyword
2. Analyzing job trends
3. Skills analysis
4. Salary insights
5. Remote vs on-site jobs

---


In [ ]:
%pip install brightdata-sdk pandas matplotlib seaborn -q

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from brightdata import BrightDataClient
from brightdata.payloads import LinkedInJobSearchPayload

sns.set_style('whitegrid')
API_TOKEN = "your_api_token_here"
client = BrightDataClient(token=API_TOKEN)
print("✅ Ready!")


## 🔍 1. Search for Jobs


In [ ]:
# Search for Python developer jobs
payload = LinkedInJobSearchPayload(
    keyword="python developer",
    location="San Francisco, CA",
    remote=True,
    experienceLevel="mid"
)

print(f"Searching for: {payload.keyword}")
print(f"Location: {payload.location}")
print(f"Remote: {payload.is_remote_search}")

result = client.search.linkedin.jobs(**payload.to_dict())

if result.success and result.data:
    jobs_df = pd.DataFrame(result.data)
    print(f"\n✅ Found {len(jobs_df)} jobs")
    print(f"Total results: {result.total_found:,}")
    display(jobs_df[['title', 'company', 'location']].head())
else:
    print(f"❌ Failed: {result.error}")


## 📊 2. Analyze Job Trends

Compare different job titles:


In [ ]:
# Compare job demand for different roles
job_titles = ["data scientist", "machine learning engineer", "data engineer"]
job_counts = []

for title in job_titles:
    result = client.search.linkedin.jobs(keyword=title, location="United States")
    if result.success:
        job_counts.append({
            'title': title,
            'count': result.total_found,
            'sample_jobs': len(result.data) if result.data else 0
        })

trends_df = pd.DataFrame(job_counts)
print("📊 Job Market Demand:")
display(trends_df)

# Visualize
plt.figure(figsize=(10, 6))
plt.bar(trends_df['title'], trends_df['count'], color=['blue', 'green', 'orange'])
plt.title('Job Market Demand by Title', fontsize=16, fontweight='bold')
plt.ylabel('Number of Job Postings')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


## 🏠 3. Remote vs On-Site Analysis


In [ ]:
# Compare remote vs on-site opportunities
remote_result = client.search.linkedin.jobs(
    keyword="python developer",
    remote=True
)

onsite_result = client.search.linkedin.jobs(
    keyword="python developer",
    location="New York, NY"
)

comparison = {
    'Remote': remote_result.total_found if remote_result.success else 0,
    'On-Site': onsite_result.total_found if onsite_result.success else 0
}

print(f"Remote jobs: {comparison['Remote']:,}")
print(f"On-site jobs: {comparison['On-Site']:,}")
print(f"Remote percentage: {100 * comparison['Remote'] / (comparison['Remote'] + comparison['On-Site']):.1f}%")


## 💾 4. Export for Further Analysis


In [ ]:
# Export job data
if len(jobs_df) > 0:
    jobs_df.to_csv('linkedin_jobs.csv', index=False)
    print("✅ Exported to linkedin_jobs.csv")
    
    # Create summary report
    summary = pd.DataFrame({
        'Metric': ['Total Jobs', 'Unique Companies', 'Remote Jobs', 'Avg Cost'],
        'Value': [
            len(jobs_df),
            jobs_df['company'].nunique() if 'company' in jobs_df else 0,
            jobs_df['remote'].sum() if 'remote' in jobs_df else 0,
            f"${result.cost:.4f}"
        ]
    })
    display(summary)


## ✅ Summary

You learned:
- ✅ Searching LinkedIn jobs with filters
- ✅ Analyzing job market trends
- ✅ Remote vs on-site comparison
- ✅ Exporting data for analysis

## 🎓 Next: [Batch Processing at Scale](./05_batch_processing.ipynb)

**Happy Job Hunting! 💼**
